In [ ]:
# Start with importing the packages
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from   sklearn.decomposition import PCA
from   sklearn.manifold import TSNE
from   sklearn.preprocessing import StandardScaler
import scipy
from scipy import stats
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
import statsmodels.stats.multitest as smm
from statsmodels.formula.api import ols

np.set_printoptions(suppress=True)
np.set_printoptions(precision=4)
plt_style = 'seaborn-notebook'
pd.set_option('precision', 2)
pd.set_option('max_columns',10)

## import data table

In [ ]:
df = pd.read_excel('ddpcr-data.xlsx')
df = df.set_index('Isolate')
print(df.shape)
df.head()

Table notes:
DVG_2A column: based on NGS data
DVG_2B column: based on ddpcr ratio, cut of at 1.2

## Stats for BK box plot

In [ ]:
## Stats for ddpcr-ratio vs DVGs
DIPYes = df.loc[df.DVG_2A == 'Yes','CopiesmL':]
DIPNo = df.loc[df.DVG_2A == 'No', 'CopiesmL':]
# test the significance of the difference
#stat, p = stats.ttest_ind(DIPYes.Final, DIPNo.Final, nan_policy='omit')
stat, p = stats.kruskal(DIPYes.Final, DIPNo.Final, nan_policy='omit')
print('p =', p)
DIPYes.describe()
DIPNo.describe()

In [ ]:
## Stats for viral-load vs DVGs
DIPYes2 = df.loc[df.DVG_2B == 'Yes','CopiesmL':]
DIPNo2 = df.loc[df.DVG_2B == 'No', 'CopiesmL':]
# test the significance of the difference0
#stat2, p2 = stats.ttest_ind(DIPYes2.log, DIPNo2.log, nan_policy='omit')
stat2, p2 = stats.kruskal(DIPYes2.log, DIPNo2.log, nan_policy='omit')
print('p value for= ', p2)
DIPYes2.describe()
DIPNo2.describe()

## Fig 2A: BK boxplot with VP/LT ratio

In [ ]:
palette = ('#000000','#FF0000')
sns.set_context("talk")
sns.set_style(style="white")
g = sns.catplot(x="DVG_2A", y="Final", data=df, hue='Notes',palette=palette)
g._legend.remove()
g = sns.boxplot(x="DVG_2A", y="Final", data=df, color = 'white',showfliers= False,linewidth=2.5)
plt.yticks(np.arange(0, 4.5, step=0.5)) 
g.set(ylim=(0.5,3.9))
plt.xlabel("DVGs Presence by Sequencing",labelpad=20)
plt.ylabel("VP:LargeT Ratio",labelpad=10)
x1, x2 = 0, 1   # columns 'Sat' and 'Sun' (first column: 0, see plt.xticks())
y, h, col = 3.7, 0.10, 'k'
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
label = "Kruskal-Wallis test, p = 0.0002"
plt.text((x1+x2)*.5, y+h+h, label, ha='center', va='bottom', color=col, fontsize=14)
#plt.plot([-0.5,1.5],[1.2,1.2],linestyle='dashed', color = 'red', linewidth = 1)
#plt.legend(fontsize="x-small", bbox_to_anchor=(1,1),loc='upper left')

#plt.tight_layout()

plt.savefig('ddpcr-fig2A_2.png', dpi=300,facecolor='white')

plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.savefig('ddpcr-fig2B_blank.png', dpi=300, facecolor='white')

## Fig 2B: BK boxplot with viral load

In [ ]:
sns.set_context("talk")
sns.set_style(style="white")
b = sns.catplot(x="DVG_2B", y="log", data=df, color='black')
#g._legend.remove()
b.set(ylim=(4,13))
b = sns.boxplot(x="DVG_2B", y="log", data=df, color = 'white',showfliers= False,linewidth=2.5)
plt.xlabel("DVGs Presence by ddPCR",labelpad=20)
plt.ylabel("log10 copies/mL",labelpad=10)
plt.yticks(np.arange(4, 13.5, step=1))
x1, x2 = 0, 1   # (first column: 0, see plt.xticks())
y, h, col = 12.5, 0.10, 'k'
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
label = "Kruskal-Wallis test, p = 0.0087"
plt.text((x1+x2)*.5, y+h+h, label, ha='center', va='bottom', color=col,fontsize=14)

#plt.legend(fontsize="x-small", bbox_to_anchor=(1,1),loc='upper left')
plt.tight_layout()

plt.savefig('ddpcr-fig2B_2.png', dpi=300, facecolor='white')

plt.show()

## Fig 2C: JC boxplot with VP/LT ratio

In [ ]:
df2 = pd.read_excel('jcddpcr-data.xlsx')
print(df2.shape)
df2

In [ ]:
DVGNo = df2.loc[df2.DVG == 'No/Unknown', 'Ratio':]
DVGNo.describe()

In [ ]:
sns.set_context("talk")
sns.set_style(style="white")
c = sns.catplot(x="DVG", y="Ratio", data=df2, color='black')
#g._legend.remove()
c = sns.boxplot(x="DVG", y="Ratio", data=df2, color = 'white',showfliers= False,linewidth=2.5)
plt.xlabel("DVGs Presence by Sequencing",labelpad=20)
plt.ylabel("VP:LargeT Ratio",labelpad=10)
plt.yticks(np.arange(0.5, 1.4, step=0.1))
#plt.legend(fontsize="x-small", bbox_to_anchor=(1,1),loc='upper left')
plt.tight_layout()
plt.savefig('ddpcr-fig2C_2.png', dpi=300)
plt.show()

## old codes

In [ ]:
palette = ('#F04DCA','#52C76D','#F78707','#3A62CF')
#palette=dict(Yes="g", No="m")
#fig33 = plt.figure()
#BK = df.loc[df.Virus2 == 'BK+','Viral_Load':]
fig33 = sns.lmplot(x="log", y="Final", data=df, scatter=False, line_kws={'color': '#848EE8',"linewidth": 1})
fig33 = sns.scatterplot(x = df['log'], y = df['Final'], data=df, hue="DVG")
fig33.set_ylabel('VP:LargeT Ratio')
fig33.set_xlabel('Log Viral Load')
#fig33.set(ylim=(0.5,4))
#plt.legend(fontsize="small", bbox_to_anchor=(0.78,1),loc='upper left', title=None)
plt.plot([4,11],[1,1], linestyle="dashed", color = 'black', linewidth = 1)
plt.show()
#plt.savefig('ddpcr-scatter-withoutngs.png')



In [ ]:
fig2 = plt.figure(figsize=(11,8))
scat = fig2.add_subplot(111)
scat.scatter(x = df.index, y = df['VP'], color='b')
scat.scatter(x = df.index, y = df['LargeT'], color='k')
scat.set_ylabel('Copies/25uL well')
scat.set_xlabel('Sample ID')
scat.set_ylim(-1,5100)
fig2 = plt.xticks(df.index, df.index, rotation='vertical')
plt.show()

In [ ]:
#BK = df.loc[df.Virus2 == 'BK+','Viral_Load':]
slope, intercept, r_value, p_value, std_err = stats.linregress(df['log'],df['Final'])
print("Stats for all data:", "Slope =",slope, "r-squared =" ,r_value**2, "p-value = ", p_value)
#fig44 = sns.lmplot(x="Log_load", y="Ratio", data=df, palette=palette, height=3, aspect=1, scatter=False)
#plt.show()